In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import uproot
import h5py as h


In [2]:
def Get_hit_array(file, Current_Event,particle):
    data = h.File(file,'r')
    TotalEvents=int(data['MC']['configuration'][2][1])
    if(TotalEvents < Current_Event):
        print('out of event range')

    Current_Hit_Mask = data['MC']['hits']['event_id'] == Current_Event 
    Current_Particle_Mask = data["MC"]['particles']['event_id'] == Current_Event
    Hits_PIDs=data['MC']['hits'][Current_Hit_Mask]['particle_id'] 
    Current_Particles=data['MC']['particles'][Current_Particle_Mask]

    Electron_Mask = Current_Particles['particle_name'] == particle

    Electron_PIDS = data['MC']['particles'][Current_Particle_Mask][Electron_Mask]['particle_id']

    Hit_Electron_Maks = np.in1d(Hits_PIDs, Electron_PIDS)
    ElectronData=data['MC']['hits'][Current_Hit_Mask][Hit_Electron_Maks]

    return ElectronData


def Diffuser(HIT_ARRAY,ZOFFSET):
    Etotal = 0
    Diffused_X = np.array([])
    Diffused_Y = np.array([])
    Diffused_Z = np.array([])

    for hit in HIT_ARRAY:
        energy_deposit = hit['energy']
        electron_loc_x = hit['x']/10
        electron_loc_y = hit['y']/10
    #     electron_loc_z = hit['z']/10 + Zoffset
        electron_loc_z = ZOFFSET

        Etotal += energy_deposit

        Nelectron = int( (energy_deposit*1e6/GasProps.Wvalue) );

        # Loop through the electrons 
        for electron in range(Nelectron):

            # calculate drift time for diffusion 
            T_drift = electron_loc_z / GasProps.Vd
            if (T_drift <=0):
                print("Warnign T_drift is negative")
                continue
            # electron lifetime
            if (np.random.uniform() >= np.exp(-T_drift/GasProps.Life_Time)):
                continue

            # diffuse the electrons position
            sigma_T = np.sqrt(2*GasProps.Dt*T_drift)
            sigma_L = np.sqrt(2*GasProps.Dl*T_drift)
            electron_x = np.random.normal(electron_loc_x,sigma_T)
            electron_y = np.random.normal(electron_loc_y,sigma_T)
            electron_z = np.random.normal(electron_loc_z,sigma_L)
            Diffused_X=np.append(Diffused_X,electron_x)
            Diffused_Y=np.append(Diffused_Y,electron_y)
            Diffused_Z=np.append(Diffused_Z,electron_z)
    print("evnet has {} MeV and made {} electrons".format(round(Etotal,3), len(Diffused_X)))
        
        
    return Diffused_X, Diffused_Y, Diffused_Z




def Diffuserv2(HIT_ARRAY,ZOFFSET):
    Etotal = 0
    Diffused_X = []
    Diffused_Y = []
    Diffused_Z = []

    for hit in HIT_ARRAY:
        energy_deposit = hit[3]
        electron_loc_x = hit[0]
        electron_loc_y = hit[1]
        #electron_loc_z = hit[2] + Zoffset
        electron_loc_z = ZOFFSET

        Etotal += energy_deposit

       
        # calculate drift time for diffusion 
        T_drift = electron_loc_z / GasProps.Vd
        if (T_drift <=0):
            print("Warnign T_drift is negative")
            continue
        # electron lifetime
        if (np.random.uniform() >= np.exp(-T_drift/GasProps.Life_Time)):
            continue

        # diffuse the electrons position
        sigma_T = np.sqrt(2*GasProps.Dt*T_drift)
        sigma_L = np.sqrt(2*GasProps.Dl*T_drift)
        electron_x = np.random.normal(electron_loc_x,sigma_T)
        electron_y = np.random.normal(electron_loc_y,sigma_T)
        electron_z = np.random.normal(electron_loc_z,sigma_L)
        Diffused_X.append(electron_x)
        Diffused_Y.append(electron_y)
        Diffused_Z.append(electron_z)
    print("evnet has {} MeV and made {} electrons".format(round(Etotal,3), len(Diffused_X)))
        
        
    return Diffused_X, Diffused_Y, Diffused_Z


def ionizationElectronPosition(data,mother,targetvolume='EL_GAP',EventLimit=2):
    TotalEvents=int(data['configuration'][2][1])
    values={}

    
    if(EventLimit>TotalEvents):
        EventLimit=TotalEvents
        
    for EventID in range(0,EventLimit):
        # obtain current event id
        EventIDMask=data["particles"]["event_id"]==EventID
        
        #store particle infor for this event
        ParticlesTable=data["particles"][EventIDMask]
        # Create Mother mask for expected mother
        MotherMask=ParticlesTable["particle_name"]==mother
        # Interested particles
        IonizationElectronsMask=ParticlesTable["final_volume"]==targetvolume
        #obtain mother ids
        IonizationElectronMotherID=ParticlesTable["mother_id"][IonizationElectronsMask]
        #Expected Mother IDs
        ExpectedMotherID=ParticlesTable["particle_id"][MotherMask]
        # Find intersection of the motherIds of ie and Expected Mother
        MotherIDMask=np.isin(IonizationElectronMotherID,ExpectedMotherID)
        FinalPositions=[ParticlesTable["final_x"][IonizationElectronsMask][MotherIDMask],ParticlesTable["final_y"][IonizationElectronsMask][MotherIDMask],ParticlesTable["final_z"][IonizationElectronsMask][MotherIDMask]]
        values[EventID]=FinalPositions
    return values
        
def getvalues(file):
    data = h.File(file,'r')
    return data['MC']
                
    
class Gas_Properties():
    # gas values for 500V/cm at 10 bar
    # simulated from PyBoltz
    def __init__(self):
        #self.Wvalue = 22.1 # in eV
        #self.Vd = 935000.0 # cm/s
        #self.Dt = 6114.291 # cm**2/s
        #self.Dl = 506.464  # cm**2/s
        #self.Life_Time = 0.001 # in s
        
        self.Wvalue = 22.1 # in eV
        self.Vd = 94000 # cm/s
        self.Dt = 395.80 # cm**2/s
        self.Dl = 61.15  # cm**2/s
        self.Life_Time = 0.001 # in s
        
        
        
GasProps = Gas_Properties()

In [3]:
### GEANT4



In [4]:
filePath="/media/ilker/Ilker/SimResults/Jan_20_2022/Alpha_Prediction.h5"


In [5]:
data=getvalues(filePath)

In [6]:
Alphaies=ionizationElectronPosition(data,b'alpha',b'EL_GAP',100)

In [7]:
for Event in Alphaies.keys():
    X=(Alphaies[Event][0])/10
    Y=Alphaies[Event][1]/10
    bins = [np.arange(-10, 10, 0.02), np.arange(-10, 10, 0.02)]
    cmap = plt.get_cmap('cividis')
    cmap = plt.get_cmap('afmhot')

    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(111)
    #ax1.set_title("GEANT4 Diffusion at 5cm")


    h1 = ax1.hist2d(X, Y, bins=bins, cmap=cmap)

    #cbar1 = plt.colorbar(h1[3])
    #cbar1.set_label('Electrons', fontsize=18)
    #cbar1.ax.tick_params(labelsize=18)
    # cbar1.ax.set_title('Electrons',fontsize=18, rotation=270)

    #ax1.set_ylabel(r'Y  [cm]',fontsize=18)
    #ax1.set_xlabel(r'X  [cm]',fontsize=18)
    #ax1.tick_params(labelsize=18)
    ax1.set_xlim(-10,10)
    ax1.set_ylim(-10,10)
    plt.axis('off')
    path="/media/ilker/Ilker/SimResults/Jan_20_2022/Particles/Predictions/Alpha/Alphas_"+str(Event)+".png"
    plt.tight_layout()
    plt.savefig(path,dpi=300,bbox_inches='tight')
    #plt.show()
    plt.close()

In [8]:
filePath="/media/ilker/Ilker/SimResults/Jan_20_2022/Betas_Prediction.h5"
data=getvalues(filePath)

In [9]:
Betas=ionizationElectronPosition(data,b'e-',b'EL_GAP',500)

In [10]:
print(Betas)


{0: [array([-10.244116 , -10.2984915,  -9.022798 , ...,  16.294918 ,
        17.338102 ,  15.978047 ], dtype=float32), array([22.403166, 18.334742, 24.096018, ..., 21.890379, 22.468437,
       20.279278], dtype=float32), array([-107.7635, -107.7635, -107.7635, ..., -107.7635, -107.7635,
       -107.7635], dtype=float32)], 1: [array([-18.343676, -22.829424, -17.670145, ...,  -9.975636, -10.417984,
       -10.409285], dtype=float32), array([3.6621323e+00, 7.7558886e-03, 1.1656480e+00, ..., 1.7361927e+01,
       1.7876617e+01, 1.8234354e+01], dtype=float32), array([-107.7635, -107.7635, -107.7635, ..., -107.7635, -107.7635,
       -107.7635], dtype=float32)], 2: [array([-6.241043  , -0.85274327, -7.238158  , ..., 24.84347   ,
       12.840421  , 18.967314  ], dtype=float32), array([ -6.590375 ,  -0.9140872,  -3.0498865, ..., -17.062431 ,
       -10.960075 , -18.058136 ], dtype=float32), array([-107.7635, -107.7635, -107.7635, ..., -107.7635, -107.7635,
       -107.7635], dtype=float32)], 

In [11]:
for Event in Betas.keys():
    X=(Betas[Event][0])/10
    Y=Betas[Event][1]/10
    bins = [np.arange(-10, 10, 0.02), np.arange(-10, 10, 0.02)]
    cmap = plt.get_cmap('cividis')
    cmap = plt.get_cmap('afmhot')

    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(111)
    #ax1.set_title("GEANT4 Diffusion at 5cm")


    h1 = ax1.hist2d(X, Y, bins=bins, cmap=cmap)

    #cbar1 = plt.colorbar(h1[3])
    #cbar1.set_label('Electrons', fontsize=18)
    #cbar1.ax.tick_params(labelsize=18)
    # cbar1.ax.set_title('Electrons',fontsize=18, rotation=270)

    #ax1.set_ylabel(r'Y  [cm]',fontsize=18)
    #ax1.set_xlabel(r'X  [cm]',fontsize=18)
    #ax1.tick_params(labelsize=18)
    ax1.set_xlim(-10,10)
    ax1.set_ylim(-10,10)
    plt.axis('off')
    path="/media/ilker/Ilker/SimResults/Jan_20_2022/Particles/Predictions/Beta/Betas_"+str(Event)+".png"
    plt.tight_layout()
    plt.savefig(path,dpi=300,bbox_inches='tight')
    #plt.show()
    plt.close()

In [12]:
#### 